In [ ]:
##第一部分领料汇总——1.项目名称匹配

import os
import pandas as pd
from fuzzywuzzy import process

# 文件路径
raw_data_file = "D:/ZHR/Requisition/2025.06/领料汇总.xlsx"
standard_names_file = "D:/ZHR/Requisition/2025.06/项目库.xlsx"
output_file_match = "D:/ZHR/Requisition/2025.06/1.项目名称匹配.xlsx"
output_file_summary = "D:/ZHR/Requisition/2025.06/2.项目金额统计.xlsx"

# 第一部分：项目名称匹配
# 如果文件被占用，生成新文件名
if os.path.exists(output_file_match):
    print(f"文件 {output_file_match} 正在被使用，将使用新文件名保存结果。")
    output_file_match = "D:/ZHR/Requisition/2025.06/1.项目名称匹配_新.xlsx"

# 读取数据
data = pd.read_excel(raw_data_file)  # 原始数据
standard_names = pd.read_excel(standard_names_file)  # 标准名数据库

# 提取标准名称列表
def get_standard_names():
    standard_names = pd.read_excel(standard_names_file)
    return standard_names["项目"].tolist(), standard_names

standard_name_list, latest_standard_names = get_standard_names()

# 定义模糊匹配函数
def match_name(name, choices, threshold=80):
    if pd.isnull(name):  # 检查是否为 NaN
        return "需人工检查", "需人工检查"
    result = process.extractOne(str(name), choices)  # 确保输入为字符串
    if result is None or result[1] < threshold:  # 没有找到足够高的匹配项
        return name, "需人工检查"
    return result[0], ""

# 应用模糊匹配并新增匹配项目列
data["匹配项目"], data["需检查"] = zip(*data["ISSUANCE WAREHOUSE"].apply(
    lambda x: match_name(x, standard_name_list)
))

# 写入匹配结果到 Excel
with pd.ExcelWriter(output_file_match, engine="openpyxl") as writer:
    data.to_excel(writer, sheet_name="项目名称匹配", index=False)

print(f"项目名称匹配结果已生成，结果保存为 {output_file_match}")

In [2]:
##第二部分领料汇总——2.项目动态小计

import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle, Alignment

# 文件路径
output_file_match = "D:/ZHR/Requisition/2025.06/1.项目名称匹配.xlsx"  # 项目名称匹配后的数据
project_library_file = "D:/ZHR/Requisition/2025.06/项目库.xlsx"  # 最新项目库文件
output_file_summary = "D:/ZHR/Requisition/2025.06/2.项目动态小计.xlsx"  # 输出统计结果

# 读取匹配后的数据
try:
    matched_data = pd.read_excel(output_file_match)
except FileNotFoundError:
    print(f"错误：文件 {output_file_match} 不存在，请检查路径或重新生成该文件。")
    exit()

# 检查匹配数据中是否包含 TOTAL AMOUNT 列
if "TOTAL AMOUNT" in matched_data.columns:
    # 读取项目库文件
    try:
        project_library = pd.read_excel(project_library_file)
    except FileNotFoundError:
        print(f"错误：文件 {project_library_file} 不存在，请检查路径或重新生成该文件。")
        exit()

    # 添加项目库的顺序信息
    project_library["序号"] = range(1, len(project_library) + 1)

    # 汇总每个项目的 TOTAL AMOUNT，并按项目库顺序合并
    total_amount_data = matched_data.groupby("匹配项目")["TOTAL AMOUNT"].sum().reset_index()
    merged_data = pd.merge(
        project_library[["序号", "项目"]],
        total_amount_data,
        left_on="项目",
        right_on="匹配项目",
        how="left"
    )
    merged_data["TOTAL AMOUNT"] = merged_data["TOTAL AMOUNT"].fillna(0).round(2)  # 未匹配项目金额设为 0，保留两位小数

    # 删除多余列，按项目库顺序保留关键列
    merged_data = merged_data[["序号", "项目", "TOTAL AMOUNT"]]
    merged_data.sort_values(by="序号", inplace=True)

    # 计算总金额
    total_amount = merged_data["TOTAL AMOUNT"].sum()

    # 在结果后添加总金额
    total_row = pd.DataFrame([["", "总金额", total_amount]], columns=["序号", "项目", "TOTAL AMOUNT"])
    merged_data = pd.concat([merged_data, total_row], ignore_index=True)

    # 输出统计结果到 Excel
    with pd.ExcelWriter(output_file_summary, engine="openpyxl") as writer:
        merged_data.to_excel(writer, sheet_name="项目动态小计", index=False)

    # 设置数值格式（千分位符、两位小数）
    wb = load_workbook(output_file_summary)
    ws = wb["项目动态小计"]

    # 定义格式
    numeric_style = NamedStyle(name="NumericStyle")
    numeric_style.number_format = "#,##0.00"  # 数值格式，千分位符，两位小数
    numeric_style.alignment = Alignment(horizontal="right")  # 右对齐

    # 应用格式到 TOTAL AMOUNT 列
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=3, max_col=3):  # 第 3 列 TOTAL AMOUNT
        for cell in row:
            cell.style = numeric_style

    # 保存结果
    wb.save(output_file_summary)
    print(f"项目动态小计已完成，结果保存为 {output_file_summary}")
else:
    print("匹配后的数据中缺少 TOTAL AMOUNT 列，无法统计。")

项目动态小计已完成，结果保存为 D:/ZHR/Requisition/2025.06/2.项目动态小计.xlsx


In [3]:
##第三部分领料汇总——3.项目成本明细

import os
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment

# 文件路径
output_file_match = "D:/ZHR/Requisition/2025.06/1.项目名称匹配.xlsx"  # 项目名称匹配后的数据
project_library_file = "D:/ZHR/Requisition/2025.06/项目库.xlsx"  # 最新项目库文件
output_file_detail = "D:/ZHR/Requisition/2025.06/3.项目成本明细.xlsx"  # 输出成本明细文件

# 读取项目匹配后的数据
try:
    matched_data = pd.read_excel(output_file_match)
except FileNotFoundError:
    print(f"错误：文件 {output_file_match} 不存在，请检查路径或重新生成该文件。")
    exit()

# 检查必需的列是否存在
required_columns = ["匹配项目", "MATERIAL NO.", "DESCRIPITION", "CHINESE NAME",
                    "SPECIFICATION", "UNIT", "QUANTITY", "UNIT PRICE", "TOTAL AMOUNT"]
missing_columns = [col for col in required_columns if col not in matched_data.columns]
if missing_columns:
    print(f"错误：缺少以下必要列：{missing_columns}，无法生成项目成本明细。")
    exit()

# 读取项目库数据
try:
    project_library = pd.read_excel(project_library_file)
except FileNotFoundError:
    print(f"错误：文件 {project_library_file} 不存在，请检查路径或重新生成该文件。")
    exit()

# 添加项目库的顺序信息
project_library["序号"] = range(1, len(project_library) + 1)

# 合并项目匹配数据和项目库，确保项目库中的项目顺序
merged_data = pd.merge(
    project_library[["序号", "项目"]],
    matched_data,
    left_on="项目",
    right_on="匹配项目",
    how="left"
)

# 确保金额列为数值型，并保留两位小数
merged_data["QUANTITY"] = merged_data["QUANTITY"].astype(float).round(2)
merged_data["UNIT PRICE"] = merged_data["UNIT PRICE"].astype(float).round(2)
merged_data["TOTAL AMOUNT"] = merged_data["TOTAL AMOUNT"].astype(float).round(2)

# 格式化为千分号分隔（Excel 自动处理）
formatted_data = merged_data[["序号", "项目", "MATERIAL NO.", "DESCRIPITION",
                              "CHINESE NAME", "SPECIFICATION", "UNIT", 
                              "QUANTITY", "UNIT PRICE", "TOTAL AMOUNT"]].copy()

# 按项目库顺序和序号排序
formatted_data.sort_values(by=["序号", "项目"], inplace=True)

# 计算生成的总金额
generated_total = formatted_data["TOTAL AMOUNT"].sum()

# 从匹配数据计算原始总金额
original_total = matched_data["TOTAL AMOUNT"].sum()

# 计算差额
difference = generated_total - original_total

# 写入 Excel 文件
with pd.ExcelWriter(output_file_detail, engine="openpyxl") as writer:
    # 写入项目成本明细
    formatted_data.to_excel(writer, sheet_name="项目成本明细", index=False)

# 加载工作簿和工作表
wb = load_workbook(output_file_detail)
ws = wb["项目成本明细"]

# 在总金额与差额下方输出
ws.append(["", "总值", "", "", "", "", "", "", "", f"{generated_total:,.2f}"])
ws.append(["", "差额", "", "", "", "", "", "", "", f"{difference:,.2f}"])

# 设置右对齐和金额格式
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=8, max_col=10):  # QUANTITY, UNIT PRICE, TOTAL AMOUNT
    for cell in row:
        cell.number_format = "#,##0.00"  # 设置金额格式
        cell.alignment = Alignment(horizontal="right")  # 右对齐

# 保存工作簿
wb.save(output_file_detail)

# 控制台输出结果
print(f"项目成本明细已生成，结果保存为 {output_file_detail}")
print(f"总值：{generated_total:,.2f}")
print(f"差额：{difference:,.2f}")

项目成本明细已生成，结果保存为 D:/ZHR/Requisition/2025.06/3.项目成本明细.xlsx
总值：2,384,759,438.92
差额：-2,384,759,438.92


In [4]:
##第四部分领料汇总——4.项目领料单

import os
import pandas as pd
from openpyxl import load_workbook, Workbook
from copy import copy
from datetime import datetime

# 文件路径
template_file = "D:/ZHR/Requisition/2025.06/领料单样.xlsx"  # 领料单样文件
project_library_file = "D:/ZHR/Requisition/2025.06/项目库.xlsx"  # 项目库文件
output_file = "D:/ZHR/Requisition/2025.06/4.项目领料单.xlsx"  # 输出的项目领料单文件

# 获取当前年月
current_date = datetime.now()
current_year = current_date.strftime("%Y")  # 年份 yyyy
current_month = current_date.strftime("%m")  # 月份 mm

# 读取项目库数据
try:
    project_library = pd.read_excel(project_library_file)
except FileNotFoundError as e:
    print(f"错误：无法读取项目库文件 - {e}")
    exit()

# 添加项目库顺序
project_library["序号"] = range(1, len(project_library) + 1)

# 创建新的 Excel 文件用于输出
wb_output = Workbook()
wb_output.remove(wb_output.active)  # 删除默认工作表

# 加载领料单样文件
wb_template = load_workbook(template_file)
ws_template = wb_template.active  # 获取模板工作表


def copy_worksheet_template(source_ws, target_ws):
    """复制工作表内容和格式"""
    # 复制单元格内容、样式
    for row in source_ws.iter_rows():
        for cell in row:
            new_cell = target_ws.cell(row=cell.row, column=cell.column, value=cell.value)
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = cell.number_format
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)

    # 复制列宽
    for col_letter, col_dimension in source_ws.column_dimensions.items():
        target_ws.column_dimensions[col_letter].width = col_dimension.width

    # 复制行高
    for row_idx, row_dimension in source_ws.row_dimensions.items():
        target_ws.row_dimensions[row_idx].height = row_dimension.height

    # 复制合并单元格
    for merged_cell_range in source_ws.merged_cells.ranges:
        target_ws.merge_cells(str(merged_cell_range))


# 遍历项目库，为每个项目生成子表
for _, project_row in project_library.iterrows():
    project_name = project_row["项目"]
    project_number = project_row["序号"]

    # 创建新子表
    ws_output = wb_output.create_sheet(title=f"{project_number}.{project_name}")

    # 复制模板内容到新子表
    copy_worksheet_template(ws_template, ws_output)

    # 修改子表中指定单元格内容
    # B1: JK-yyyymmnn
    ws_output["B1"] = f"JK-{current_year}{current_month}{project_number:02d}"
    # I1: 项目名称
    ws_output["I1"] = project_name
    # I2: 项目序号
    ws_output["I2"] = project_number

# 保存输出文件
wb_output.save(output_file)
print(f"项目领料单已生成，结果保存为 {output_file}")


项目领料单已生成，结果保存为 D:/ZHR/Requisition/2025.06/4.项目领料单.xlsx


In [12]:
##第五部分领料汇总——5.填充领料单
import openpyxl
from openpyxl.worksheet.worksheet import Worksheet

def set_print_area_with_preview(file_path, output_path, print_area="A1:I11"):
    # 打开 Excel 文件
    wb = openpyxl.load_workbook(file_path)
    
    # 遍历所有工作表
    for sheet in wb.worksheets:
        sheet.print_area = print_area  # 设置打印区域
        sheet.page_setup.orientation = "portrait"  # 设置为纵向
        sheet.page_setup.fitToPage = True  # 适应页面
        sheet.page_setup.fitToHeight = 1  # 调整高度以适应单页
        sheet.page_setup.fitToWidth = 1  # 调整宽度以适应单页
        sheet.sheet_view.view = "pageBreakPreview"  # 设置为分页预览模式
    
    # 保存为新文件
    wb.save(output_path)
    print(f"成功设置文件 {file_path} 中所有工作表的打印区域为 {print_area}，并保持分页预览模式，保存为 {output_path}")

# 指定文件路径和输出路径
file_path = "D:/ZHR/Requisition/2025.06/4.项目领料单.xlsx"
output_path = "D:/ZHR/Requisition/2025.06/5.填充领料单.xlsx"
set_print_area_with_preview(file_path, output_path)

成功设置文件 D:/ZHR/Requisition/2025.06/4.项目领料单.xlsx 中所有工作表的打印区域为 A1:I11，并保持分页预览模式，保存为 D:/ZHR/Requisition/2025.06/5.填充领料单.xlsx
